In [1]:
import math

In [2]:
# Define a pace data structure
class Pace():
    def __init__(self, *args):
        arg_len = len(args)
        if arg_len == 1:
            if isinstance(args[0], str):
                args = args[0].split(":")
                arg_len = len(args)
            else:
                raise ValueError("Pace must be initialized with at least seconds")
        elif arg_len == 2:
            self.minutes = args[0]
            self.seconds = args[1]
            self.tenths = 0
            self.hours = 0
        elif arg_len == 3:
            self.hours = 0
            self.minutes = args[0]
            self.seconds = args[1]
            self.tenths = args[2]
        elif arg_len == 4:
            self.hours = args[0]
            self.minutes = args[1]
            self.seconds = args[2]
            self.tenths = args[3]
        self.dist_unit = "km"

    def __add__(self, other):
        _return = Pace(self.hours, self.minutes, self.seconds, self.tenths) 
        if _return.tenths != 0 or other.tenths != 0:
            _return.tenths += other.tenths
            if _return.tenths >= 60:
                _return.seconds += 1
                _return.tenths -= 60
        _return.seconds += other.seconds 
        if _return.seconds >= 60:
            _return.minutes += 1
            _return.seconds -= 60
        _return.minutes += other.minutes
        if _return.minutes >= 60:
            _return.hours += 1
            _return.minutes -= 60
        if (_return.hours != 0) or (other.hours != 0):
            _return.hours += other.hours
            
        return _return
    
    def __sub__(self, other):
        _return = Pace(self.hours, self.minutes, self.seconds, self.tenths) 
        if _return.tenths != 0 or other.tenths != 0:
            _return.tenths -= other.tenths
            if _return.tenths < 0:
                _return.seconds -= 1
                _return.tenths += 60
        _return.seconds -= other.seconds
        if _return.seconds < 0:
            _return.minutes -= 1
            _return.seconds += 60
        _return.minutes -= other.minutes
        if _return.minutes < 0 and self.hours > 0:
            _return.hours -= 1
            _return.minutes += 60
        elif _return.minutes < 0 and self.hours == 0:
            raise ValueError("Can't have negative pace")
        if (_return.hours != 0) or (other.hours != 0):
            if _return.hours == 0:
                _return.hours = 0
            if other.hours == 0:
                raise ValueError("Can't have negative pace")
        return _return
    
    def __mul__(self, scalar):
        _return = Pace(self.hours, self.minutes, self.seconds, self.tenths)
        if scalar < 0:
            raise ValueError("Can't have negative pace")
        _return.tenths *= scalar
        _return.seconds *= scalar
        _return.minutes *= scalar
        _return.hours *= scalar
        _keep_going = True
        while _keep_going:
            if _return.tenths >= 60:
                _return.seconds += 1
                _return.tenths -= 60
            if _return.seconds % 1 > 0:
                _return.tenths += round((_return.seconds % 1) * 60, 2)
                _return.seconds = int(_return.seconds // 1)
            _return.tenths = int(_return.tenths // 1)
            if _return.seconds >= 60:
                _return.minutes += 1
                _return.seconds -= 60
            if _return.minutes % 1 > 0:
                _return.seconds += round((_return.minutes % 1) * 60, 2)
                _return.minutes = int(_return.minutes // 1)
            _return.seconds = int(_return.seconds // 1)
            if _return.minutes >= 60:
                _return.hours += 1
                _return.minutes -= 60
            if _return.hours % 1 > 0:
                _return.minutes += round((_return.hours % 1) * 60, 2)
            _return.minutes = int(_return.minutes // 1)
            _return.hours = int(_return.hours // 1)
            if _return.tenths < 60 and _return.seconds < 60 and _return.minutes < 60:
                _keep_going = False
        return _return
    
    def __truediv__(self, scalar):
        if scalar == 0:
            raise ValueError("Can't divide by zero")
        scalar = 1/scalar
        return self.__mul__(scalar)

    def __str__(self):
        _str = f"{self.minutes:02d}:{self.seconds:02d}"
        _unit = f"min:sec"
        if self.tenths != 0:
            _str += f":{self.tenths:02d}"
            _unit += ":tenths"
        if self.hours != 0:
            _str = f"{self.hours:02d}:" + _str
            _unit = f"hours:" + _unit
        _unit += f"/{self.dist_unit}"
        return _str+" "+_unit
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        return (self.hours == other.hours) and (self.minutes == other.minutes) and (self.seconds == other.seconds) and (self.tenths == other.tenths)



In [3]:
# Test Add and Subtract

new_pace = Pace(5, 30) + Pace(5, 45)
assert new_pace == Pace(11, 15), f"Got unexpected result {new_pace}"
print(f"Got expected result {new_pace}")

try:
    Pace(5, 30) - Pace(5, 45)
except ValueError as e:
    assert str(e) == "Can't have negative pace"
    print("Got expected error")

new_pace = Pace(5, 30) - Pace(5, 15)
assert new_pace == Pace(0, 15), f"Got unexpected result {new_pace}"
print(f"Got expected result {new_pace}")


Got expected result 11:15 min:sec/km
Got expected error
Got expected result 00:15 min:sec/km


In [4]:
# Test Multiply

new_pace = Pace(5, 30) * 2
assert new_pace == Pace(11, 0), f"Got unexpected result {new_pace}"
print(f"Got expected result {new_pace}")

new_pace = Pace(5, 00) * 1.5
assert new_pace == Pace(7, 30), f"Got unexpected result {new_pace}"
print(f"Got expected result {new_pace}")

new_pace = Pace(4, 12) * 0.8
assert new_pace == Pace(3, 21, 36), f"Got unexpected result {new_pace}"
print(f"Got expected result {new_pace}")

# Test Divide
new_pace = Pace(4, 12) / 2
assert new_pace == Pace(2, 6), f"Got unexpected result {new_pace}"
print(f"Got expected result {new_pace}")

Got expected result 11:00 min:sec/km
Got expected result 07:30 min:sec/km
Got expected result 03:21:36 min:sec:tenths/km
Got expected result 02:06 min:sec/km


In [5]:
class RunPredictor():
    def __init__(self, 
                 base_pace=Pace(5, 0),
                 base_dist=10,
                 dist_factor=1.2):
        # Establish run factors

        # Base Pace is a comfortable pace for a 10k
        self.base_pace = base_pace
        self.base_dist = base_dist

        # Scalar factors
        # Distance factor is a linear response to the distance of the run
        self.scalar_factors = {
            "dist_factor": dist_factor
        }

    def calculate_dist_adjustment(self, distance):
        distance_multiple = distance / self.base_dist
        return  math.pow(self.scalar_factors["dist_factor"], distance_multiple - 1)

    def predict(self, distance):
        _dist_mod = self.calculate_dist_adjustment(distance)
        return self.base_pace * _dist_mod

In [6]:
test_5_10 = RunPredictor(base_pace=Pace(5,30), base_dist=21.1, dist_factor=1.3)
print(test_5_10.predict(5))
print(test_5_10.predict(10))
print(test_5_10.predict(15))
print(test_5_10.predict(21.1))
print(test_5_10.predict(42.2))
print(test_5_10.predict(50))


04:29:33 min:sec:tenths/km
04:47:07 min:sec:tenths/km
05:05:48 min:sec:tenths/km
05:30 min:sec/km
07:09 min:sec/km
07:51:58 min:sec:tenths/km


In [7]:
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
comfortable_paces = [
    [Pace(4, 30), 5],
    [Pace(5, 0), 10],
    [Pace(5, 30), 15],
    [Pace(6, 0), 21.1],
]

plt.plot([x[1] for x in comfortable_paces], [x[0].minutes for x in comfortable_paces])